In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [2]:
ob_df = pd.read_csv('/Users/mac/Downloads/아워박스_통합관리_조회_초안 - 시트1 (6).csv')
fs_df = pd.read_excel('/Users/mac/Downloads/주문내역_2024-06-09~2025-06-13.xlsx')
code_df = pd.read_csv('/Users/mac/Downloads/store_t_product_option_export_2025-06-12_174426.csv')

In [3]:
ob_df.index.size

215

In [4]:
# ob_df의 주문서 상태의 html 코드 없애기
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("<span class='square square-primary'>","")
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("</span>","")
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("<span class='square square-warning'>","")
ob_df['주문서상태'].unique().tolist()

['반품', '정상']

In [5]:
# ob_df의 품목명 상태의 html 코드 없애기
ob_df['품목명'] = ob_df['품목명'].apply(lambda x: x.split('</span> ')[1].split('</')[0].strip())
ob_df['품목명'].unique().tolist()

['[YDY] 액티브헴철 (8EA)',
 '[YDY] 액티브헴철 (3EA)',
 '[YDY] 퓨어 리포좀 비타민C (6EA)',
 '[YDY] 액티브비큐텐',
 '[YDY] 퓨어 리포좀 비타민C (3EA)',
 '[YDY] 폴라초임계오메가3',
 '[YDY] 프로바이오 에스엘비(SLB) (2EA)',
 '[YDY] 프로바이오 에스엘비(SLB) (4EA)',
 '[YDY] 액티브헴철 (6EA)',
 '[YDY] 콜라겐 비오틴 듀오',
 '[YDY] 프로바이오 에스엘비(SLB) (6EA)',
 '[YDY] 퓨어 리포좀 비타민C (12EA)',
 '[YDY] 포스트바이오틱스 GX',
 '[YDY] 코랄칼마디 K2_R1',
 '[YDY] 프로바이오 에스엘비(SLB)',
 '[YDY] 액티브헴철 (12EA)',
 '[YDY] 썬 비타민D3 & K2_R1',
 '[YDY] 마그듀오 K2_R1',
 '[YDY] 퓨어 리포좀 비타민C',
 '[파이토알엑스] 이뮨 빌베리 (3EA)',
 '[파이토알엑스] 이뮨 빌베리 (6EA)',
 '[파이토알엑스] 이뮨 빌베리 (12EA)',
 '[파이토알엑스] 이뮨 빌베리 (8EA)',
 '[YDY] 퓨어 리포좀 글루타치온',
 '[YDY] 내몸에 효소가득',
 '[YDY] 파이토 레드큐민 (3EA)',
 '[YDY] 파이토 레드큐민 (12EA)',
 '[YDY] 오큐클리어 (6EA)',
 '[YDY] 오큐클리어 (3EA)',
 '[YDY] 퓨어 레드큐민 (3EA)',
 '[YDY] 액티브헴철',
 '[파이토알엑스] CBP 락토페린 플러스',
 '[YDY] 오큐클리어',
 '[YDY] 오마비 세트(폴라초임계오메가3 (1EA) + 마그듀오 (1EA) + 액티브비큐텐 (1EA)) + 쇼핑백 (1EA)']

In [6]:
# fs_df 에서 테스트 상품은 제거
fs_df = fs_df.drop(fs_df[fs_df['상품명'].str.contains('테스트')].index)
fs_df = fs_df.drop(fs_df[fs_df['상품명'].str.contains('QA')].index)
fs_df = fs_df.drop(fs_df[fs_df['옵션명'].str.contains('QA')].index)

In [7]:
temp_df = pd.merge(fs_df, code_df[['option_name', 'fulfillment_item_code']], left_on='옵션명', right_on='option_name', how='left')
temp_df.loc[temp_df['fulfillment_item_code'].isna(), '옵션명'].unique().tolist()


temp_df.loc[temp_df['옵션명'] == '마그듀오', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '코랄칼마디', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '썬비타민D3&K2', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/형제약사] YDY 썬비타민D3 2500IU 1박스(60일분)', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/친한] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/친한] YDY 썬비타민D3 2500IU 1박스(60일분)', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/친한] YDY 마그듀오 1박스(1개월분)', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/고상온약사] YDY 썬비타민D3 2500IU 1박스(60일분)', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/위키약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/약사임당] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/고상온약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/남기헌약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/이재연약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/안재돈약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/고상온약사] YDY 마그듀오 1박스(1개월분)', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/남기헌약사] YDY 마그듀오 1박스(1개월분)', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '풀스펙멀티비타민', 'fulfillment_item_code'] = 201067000013

fs_df = temp_df

In [8]:
null_df = fs_df[fs_df['fulfillment_item_code'].isna()]
null_df['상품명'].unique()

array(['[공동구매] 자신약사 X 파이토알엑스 이뮨 빌베리 / 3박스 6박스 8박스 12박스',
       '[공동구매] 은하약사 X YDY 액티브헴철 / 3박스 6박스  12박스',
       '[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스',
       '[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5 8박스',
       '[공동구매] 몸짱약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 6박스'], dtype=object)

In [9]:
fs_df.loc[fs_df['fulfillment_item_code'].isna(), 'fulfillment_item_code'] = 0

In [10]:
fs_df['fulfillment_item_code'] = fs_df['fulfillment_item_code'].astype(int)

In [11]:
fs_df['결제 완료일'] = fs_df['결제 완료일'].dt.strftime('%Y-%m-%d %H:%M:%S')
ob_df['주문일시'] = pd.to_datetime(ob_df['주문일시'])
ob_df['주문일시'] = ob_df['주문일시'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [12]:
ob_df.index.size

215

In [13]:
fs_df = fs_df[fs_df.duplicated(subset=['결제 완료일', 'fulfillment_item_code'], keep=False)]

In [14]:
temp_df = pd.merge(ob_df, fs_df[['결제 완료일', 'fulfillment_item_code', '옵션명', '상품명', '상품 번호']], left_on=['주문일시','업체품목코드'], right_on=['결제 완료일','fulfillment_item_code'], how='inner')

In [15]:
temp_df['도서산간여부'] = temp_df['도서산간여부'].apply(lambda x: x.split("square-gray'>")[1].split('<')[0]) 

In [16]:
temp_df['채널상품명'] = temp_df['상품명']
temp_df['옵션1'] = temp_df['옵션명']
temp_df['채널아이디'] = temp_df['상품 번호']

del temp_df['결제 완료일']
del temp_df['fulfillment_item_code']
del temp_df['옵션명']
del temp_df['상품명']
del temp_df['상품 번호']

In [17]:
types = []
temp_df.loc[temp_df['채널상품명'].isna(), '채널상품명'] = '없음'

for i in temp_df['채널상품명']:
    if '공동' in i:
        types.append('공동구매')
    elif '스페셜' in i:
        types.append('친한구매')
    elif '임직' in i:
        types.append('임직원몰 상품')
    else:
        types.append('없음')

temp_df['구분'] = types

In [18]:
temp_df['주문번호'] = temp_df['주문번호'].apply(lambda x: x[:5])

In [19]:
temp_df.index.size

361

In [20]:
temp_df.to_excel(f'/Users/mac/Downloads/아워박스_업데이트_{datetime.now()}.xlsx')